![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Grouping Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

## Goal 1: Summarize

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

### Table columns view
![table example](img/columns.png)

### Terminology

- Schema
- Primary Key
- Foreign Key
- Structured queries
- Views

***
### SQLite

![sqlite](img/SQLite-Python.jpg)

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to an Oracle database server there is a different package, `cx_Oracle`. The documentation for that package is [here](https://oracle.github.io/python-cx_Oracle/)

***
## Goal 3: Get going with sqlite!

In [1]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [2]:
con = sqlite3.connect('flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [3]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('airports',), ('airlines',), ('routes',)]


#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [4]:
cursor.execute("SELECT * FROM airports")

#### Use description

In [5]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [6]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'country', 'TEXT', 0, None, 0)
(5, 'code', 'TEXT', 0, None, 0)
(6, 'icao', 'TEXT', 0, None, 0)
(7, 'latitude', 'TEXT', 0, None, 0)
(8, 'longitude', 'TEXT', 0, None, 0)
(9, 'altitude', 'TEXT', 0, None, 0)
(10, 'offset', 'TEXT', 0, None, 0)
(11, 'dst', 'TEXT', 0, None, 0)
(12, 'timezone', 'TEXT', 0, None, 0)


In [7]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'alias', 'TEXT', 0, None, 0)
(4, 'iata', 'TEXT', 0, None, 0)
(5, 'icao', 'TEXT', 0, None, 0)
(6, 'callsign', 'TEXT', 0, None, 0)
(7, 'country', 'TEXT', 0, None, 0)
(8, 'active', 'TEXT', 0, None, 0)


In [8]:
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'airline', 'TEXT', 0, None, 0)
(2, 'airline_id', 'TEXT', 0, None, 0)
(3, 'source', 'TEXT', 0, None, 0)
(4, 'source_id', 'TEXT', 0, None, 0)
(5, 'dest', 'TEXT', 0, None, 0)
(6, 'dest_id', 'TEXT', 0, None, 0)
(7, 'codeshare', 'TEXT', 0, None, 0)
(8, 'stops', 'TEXT', 0, None, 0)
(9, 'equipment', 'TEXT', 0, None, 0)


#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

**Select**

**From**

**Where**

**Order by**

**Limit**

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

In [15]:
cursor.execute('''SELECT name
                    FROM airlines
                    WHERE active = 'Y'
                    AND country = 'United Kingdom';''').fetchall()

[('Astraeus',),
 ('Air Southwest',),
 ('Aurigny Air Services',),
 ('Air Wales',),
 ('AD Aviation',),
 ('Air Foyle',),
 ('British Airways',),
 ('British International Helicopters',),
 ('bmi',),
 ('bmibaby',),
 ('British Midland Regional',),
 ('British Mediterranean Airways',),
 ('BA CityFlyer',),
 ('Crest Aviation',),
 ('Eastern Airways',),
 ('Excel Airways',),
 ('Excel Charter',),
 ('easyJet',),
 ('First Choice Airways',),
 ('Flightline',),
 ('Flybe',),
 ('Flyglobespan',),
 ('GB Airways',),
 ('Highland Airways',),
 ('Jet2.com',),
 ('Monarch Airlines',),
 ('MyTravel Airways',),
 ('Norfolk County Flight College',),
 ('Kinloss Flying Training Unit',),
 ('ScotAirways',),
 ('Thomas Cook Airlines',),
 ('Thomsonfly',),
 ('Virgin Atlantic Airways',),
 ('Jc royal.britannica',),
 ('Royal European Airlines',),
 ('CB Airways UK ( Interliging Flights )',),
 ('Air Cudlua',),
 ('BBN-Airways',),
 ('Rainbow Air Euro',),
 ('All Europe',)]

In [19]:
cursor.execute('''SELECT DISTINCT country
                    FROM airports
                    ORDER BY country ASC ;''').fetchall()

[('Afghanistan',),
 ('Albania',),
 ('Algeria',),
 ('American Samoa',),
 ('Angola',),
 ('Anguilla',),
 ('Antarctica',),
 ('Antigua and Barbuda',),
 ('Argentina',),
 ('Armenia',),
 ('Aruba',),
 ('Australia',),
 ('Austria',),
 ('Azerbaijan',),
 ('Bahamas',),
 ('Bahrain',),
 ('Bangladesh',),
 ('Barbados',),
 ('Belarus',),
 ('Belgium',),
 ('Belize',),
 ('Benin',),
 ('Bermuda',),
 ('Bhutan',),
 ('Bolivia',),
 ('Bosnia and Herzegovina',),
 ('Botswana',),
 ('Brazil',),
 ('British Indian Ocean Territory',),
 ('British Virgin Islands',),
 ('Brunei',),
 ('Bulgaria',),
 ('Burkina Faso',),
 ('Burma',),
 ('Burundi',),
 ('Cambodia',),
 ('Cameroon',),
 ('Canada',),
 ('Cape Verde',),
 ('Cayman Islands',),
 ('Central African Republic',),
 ('Chad',),
 ('Chile',),
 ('China',),
 ('Christmas Island',),
 ('Cocos (Keeling) Islands',),
 ('Colombia',),
 ('Comoros',),
 ('Congo (Brazzaville)',),
 ('Congo (Kinshasa)',),
 ('Cook Islands',),
 ('Costa Rica',),
 ("Cote d'Ivoire",),
 ('Croatia',),
 ('Cuba',),
 ('Cyprus

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

![venn](img/venn.png)

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [43]:
import pandas as pd

# cursor.execute('''SELECT latitude,longitude
#                     FROM airports
#                     LEFT JOIN routes
#                     ON 'airline_id';''').fetchall()
# print()

# df = pd.read_sql('''SELECT latitude,longitude
#                     FROM airports
#                     LEFT JOIN routes
#                     ON 'airline_id';''',con)

df = pd.read_sql('''SELECT routes.airline_id, airports.longitude, airports.latitude, routes.*
                    FROM routes
                    JOIN airports ON airports.id = routes.source_id
                    ;''',con)

df.head()

,airline_id,longitude,latitude,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,410,39.956589,43.449928,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,410,48.006278,46.283333,1,2B,410,ASF,2966,KZN,2990,None,0,CR2
2,410,48.006278,46.283333,2,2B,410,ASF,2966,MRV,2962,None,0,CR2
3,410,61.503333,55.305836,3,2B,410,CEK,2968,KZN,2990,None,0,CR2
4,410,61.503333,55.305836,4,2B,410,CEK,2968,OVB,4078,None,0,CR2


In [31]:
df = pd.read_sql('''SELECT *
                    FROM routes
                    ;''',con)

df.head()

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2


### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airports?
- Which countries have the highest amount of inactive airports?
- What about airports by timezones?

## Goal 5: Using sql within pandas to filter

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [ ]:
# Get data for an example
import pandas as pd
shelter_data=pd.read_csv('https://data.austintexas.gov/api/views/9t4d-g238/rows.csv?accessType=DOWNLOAD')
shelter_data.dtypes

In [ ]:
max_data = shelter_data.query('Name == "Max"')
max_data.head()

In [ ]:
shelter_data.rename(index=str, columns={"Animal Type": "animal_type"}, inplace = True)
test = shelter_data.query('animal_type == "Dog"')
test.head()

## Goal 6: Transfering from sqlite to pandas

In [ ]:
conn = sqlite3.connect("flights.db")
df = pd.read_sql_query("select * from airports limit 5;", conn)
df

**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

## Integration

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

In [ ]:
To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [91]:
# Question 1 How many tables are in the database?

conn = sqlite3.connect('Chinook_Sqlite.sqlite')
c = conn.cursor()

c.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = c.fetchall()
print(tables)
print(len(tables))

# There are 11 tables in the database.

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
11


In [ ]:
# Question 1 How many tables are in the database?

conn = sqlite3.connect('Chinook_Sqlite.sqlite')
c = conn.cursor()

c.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = c.fetchall()
print(tables)
print(len(tables))

# There are 11 tables in the database.

# Question 2 What's the primary key of each table?


listo = []
for table in tables:
    df = pd.read_sql('SELECT * FROM ' + str(table[0]), conn)
    listo.append(df.columns[0])
print(listo)

# for table in tables:
#     c.execute("PRAGMA table_info(" + table[0] + ")"
#     info = c.fetchall()
#     keys.append(info[0][1])

# print(keys)

# keys = []
# df = pd.read_sql_query('SELECT * FROM track', conn)
# df

# Question 3 What foreign keys join the tables together?
print(c.execute('''PRAGMA foreign_key_list('Album');''').fetchall()) # 
print(c.execute('''PRAGMA foreign_key_list('Genre');''').fetchall())
print(c.execute('''PRAGMA foreign_key_list('Artist');''').fetchall())
print(c.execute('''PRAGMA foreign_key_list('Customer');''').fetchall())
print(c.execute('''PRAGMA foreign_key_list('Employee');''').fetchall())
print(c.execute('''PRAGMA foreign_key_list('Invoice');''').fetchall())
print(c.execute('''PRAGMA foreign_key_list('InvoiceLine');''').fetchall())
print(c.execute('''PRAGMA foreign_key_list('MediaType');''').fetchall())
print(c.execute('''PRAGMA foreign_key_list('Playlist');''').fetchall())
print(c.execute('''PRAGMA foreign_key_list('PlaylistTrack');''').fetchall())
print(c.execute('''PRAGMA foreign_key_list('Track');''').fetchall())

# Question 4 If you had to draw a schema of how the tables are connected, what would it look like?

# Question 5 What are the max and min dates in the Invoice table?
print(c.execute('''SELECT MAX(InvoiceDate) FROM Invoice;''').fetchall())
print(c.execute('''SELECT MIN(InvoiceDate) FROM Invoice;''').fetchall())

# df = pd.read_sql('''SELECT * FROM INVOICE;''', con = conn)
# df

# Question 6 What tables would you need to answer "what is your most popular track?"

# df = pd.read_sql('''SELECT * FROM Track;''', con=conn)
# df
# df1 = pd.read_sql('''SELECT * FROM PlaylistTrack;''', con=conn)
# df1
# Use the TrackID from Track table to figure out which tracks are most popular in the PlaylistTrack table.

# Question 7 What values from each table?

df = pd.read_sql('''SELECT * )''')

In [168]:
# Question 3 What foreign keys join the tables together?
print(c.execute('''PRAGMA foreign_key_list('Album');''').fetchall()) # 
print(c.execute('''PRAGMA foreign_key_list('Genre');''').fetchall())
print(c.execute('''PRAGMA foreign_key_list('Artist');''').fetchall())
print(c.execute('''PRAGMA foreign_key_list('Customer');''').fetchall())
print(c.execute('''PRAGMA foreign_key_list('Employee');''').fetchall())
print(c.execute('''PRAGMA foreign_key_list('Invoice');''').fetchall())
print(c.execute('''PRAGMA foreign_key_list('InvoiceLine');''').fetchall())
print(c.execute('''PRAGMA foreign_key_list('MediaType');''').fetchall())
print(c.execute('''PRAGMA foreign_key_list('Playlist');''').fetchall())
print(c.execute('''PRAGMA foreign_key_list('PlaylistTrack');''').fetchall())
print(c.execute('''PRAGMA foreign_key_list('Track');''').fetchall())

[(0, 0, 'Artist', 'ArtistId', 'ArtistId', 'NO ACTION', 'NO ACTION', 'NONE')]
[]
[]
[(0, 0, 'Employee', 'SupportRepId', 'EmployeeId', 'NO ACTION', 'NO ACTION', 'NONE')]
[(0, 0, 'Employee', 'ReportsTo', 'EmployeeId', 'NO ACTION', 'NO ACTION', 'NONE')]
[(0, 0, 'Customer', 'CustomerId', 'CustomerId', 'NO ACTION', 'NO ACTION', 'NONE')]
[(0, 0, 'Track', 'TrackId', 'TrackId', 'NO ACTION', 'NO ACTION', 'NONE'), (1, 0, 'Invoice', 'InvoiceId', 'InvoiceId', 'NO ACTION', 'NO ACTION', 'NONE')]
[]
[]
[(0, 0, 'Track', 'TrackId', 'TrackId', 'NO ACTION', 'NO ACTION', 'NONE'), (1, 0, 'Playlist', 'PlaylistId', 'PlaylistId', 'NO ACTION', 'NO ACTION', 'NONE')]
[(0, 0, 'MediaType', 'MediaTypeId', 'MediaTypeId', 'NO ACTION', 'NO ACTION', 'NONE'), (1, 0, 'Genre', 'GenreId', 'GenreId', 'NO ACTION', 'NO ACTION', 'NONE'), (2, 0, 'Album', 'AlbumId', 'AlbumId', 'NO ACTION', 'NO ACTION', 'NONE')]


In [ ]:
# Question 4 If you had to draw a schema of how the tables are connected, what would it look like?

In [167]:
# Question 5 What are the max and min dates in the Invoice table?
print(c.execute('''SELECT MAX(InvoiceDate) FROM Invoice;''').fetchall())
print(c.execute('''SELECT MIN(InvoiceDate) FROM Invoice;''').fetchall())

# df = pd.read_sql('''SELECT * FROM INVOICE;''', con = conn)
# df

[('2013-12-22 00:00:00',)]
[('2009-01-01 00:00:00',)]


In [171]:
# Question 6 What tables would you need to answer "what is your most popular track?"

# df = pd.read_sql('''SELECT * FROM Track;''', con=conn)
# df
# df1 = pd.read_sql('''SELECT * FROM PlaylistTrack;''', con=conn)
# df1
# Use the TrackID from Track table to figure out which tracks are most popular in the PlaylistTrack table.

In [172]:
# Question 7 What values from each table?

df = pd.read_sql('''SELECT * )''')

## Reflection